In [117]:
import pandas as pd
import pathlib
import math
from collections import defaultdict

ALL_ZONES = "zonas_lat_long.csv"
ALL_VOTES = "resultado_presidente.csv"

zones = pd.read_csv(ALL_ZONES, encoding="latin1") #, index_col="numero_zona")
votes = pd.read_csv(ALL_VOTES, encoding="latin1", delimiter=";", \
                    usecols=["NR_TURNO", "SG_UF", "NR_ZONA", "NR_SECAO", \
                            "NM_VOTAVEL", "QT_VOTOS"])

# Ignore international votes
votes = votes[votes["SG_UF"] != "ZZ"]

In [118]:
zones.head()

,numero_zona,sigla_uf.1,lat_0,lng_0
0,60,MT,-13.662917,-57.888985
1,36,MT,-12.290314,-55.295626
2,20,MT,-15.647176,-56.130580
3,49,MT,-15.647176,-56.130580
4,26,MT,-14.667030,-52.352839


In [124]:
def save_merged_results(votes, zones, numero_turno="both", drop_na=True):
    if numero_turno != "both":
        votes = votes[votes["NR_TURNO"] == numero_turno]
    merged_data = votes.merge(zones, left_on=["SG_UF", "NR_ZONA"], \
                              right_on=["sigla_uf.1", "numero_zona"], how="right")
    idxs = merged_data["lat_0"].isna()
    if sum(idxs):
        print(f"There were {sum(idxs)} (={sum(idxs)/len(merged_data):.2f}%) "\
            "rows with invalid state initials/zone")
        if drop_na:
            merged_data = merged_data[~idxs]
    
    return merged_data

In [129]:
df = save_merged_results(votes, zones, numero_turno=1)
df.to_csv("Voting_lat_long.csv", index=False)

There were 1298 (=0.00%) rows with invalid state initials/zone


In [130]:
df

,NR_TURNO,SG_UF,NR_ZONA,NR_SECAO,NM_VOTAVEL,QT_VOTOS,numero_zona,sigla_uf.1,lat_0,lng_0
0,1.0,RS,115.0,57.0,VOTO BRANCO,1.0,115,RS,-28.291319,-53.499141
1,1.0,RS,115.0,32.0,JOï¿½O DIONISIO FILGUEIRA BARRETO AMOEDO,4.0,115,RS,-28.291319,-53.499141
2,1.0,RS,115.0,104.0,FERNANDO HADDAD,91.0,115,RS,-28.291319,-53.499141
3,1.0,RS,115.0,11.0,VOTO NULO,9.0,115,RS,-28.291319,-53.499141
4,1.0,RS,115.0,47.0,FERNANDO HADDAD,38.0,115,RS,-28.291319,-53.499141
5,1.0,RS,115.0,10.0,VOTO BRANCO,2.0,115,RS,-28.291319,-53.499141
6,1.0,RS,115.0,93.0,VOTO NULO,6.0,115,RS,-28.291319,-53.499141
7,1.0,RS,115.0,38.0,ALVARO FERNANDES DIAS,1.0,115,RS,-28.291319,-53.499141
8,1.0,RS,115.0,43.0,ALVARO FERNANDES DIAS,3.0,115,RS,-28.291319,-53.499141
9,1.0,RS,115.0,95.0,VOTO BRANCO,15.0,115,RS,-28.291319,-53.499141


In [135]:
for a, b in df.groupby(by=["SG_UF", "NR_ZONA", "NR_SECAO"]):
    print(b)
    break

       NR_TURNO SG_UF  NR_ZONA  NR_SECAO              NM_VOTAVEL  QT_VOTOS  \
53364       1.0    AC      1.0       4.0  JAIR MESSIAS BOLSONARO     167.0   

       numero_zona sigla_uf.1     lat_0     lng_0  
53364            1         AC -9.972963 -67.85329  


In [136]:
df

,NR_TURNO,SG_UF,NR_ZONA,NR_SECAO,NM_VOTAVEL,QT_VOTOS,numero_zona,sigla_uf.1,lat_0,lng_0
0,1.0,RS,115.0,57.0,VOTO BRANCO,1.0,115,RS,-28.291319,-53.499141
1,1.0,RS,115.0,32.0,JOï¿½O DIONISIO FILGUEIRA BARRETO AMOEDO,4.0,115,RS,-28.291319,-53.499141
2,1.0,RS,115.0,104.0,FERNANDO HADDAD,91.0,115,RS,-28.291319,-53.499141
3,1.0,RS,115.0,11.0,VOTO NULO,9.0,115,RS,-28.291319,-53.499141
4,1.0,RS,115.0,47.0,FERNANDO HADDAD,38.0,115,RS,-28.291319,-53.499141
5,1.0,RS,115.0,10.0,VOTO BRANCO,2.0,115,RS,-28.291319,-53.499141
6,1.0,RS,115.0,93.0,VOTO NULO,6.0,115,RS,-28.291319,-53.499141
7,1.0,RS,115.0,38.0,ALVARO FERNANDES DIAS,1.0,115,RS,-28.291319,-53.499141
8,1.0,RS,115.0,43.0,ALVARO FERNANDES DIAS,3.0,115,RS,-28.291319,-53.499141
9,1.0,RS,115.0,95.0,VOTO BRANCO,15.0,115,RS,-28.291319,-53.499141


In [137]:
temp = pd.read_csv(ALL_VOTES, encoding="latin1", delimiter=";")

In [140]:
temp.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'NR_SECAO', 'CD_CARGO', 'DS_CARGO',
       'NR_VOTAVEL', 'NM_VOTAVEL', 'QT_VOTOS'],
      dtype='object')